In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from scipy import stats
import statistics
import itertools

In [2]:
simplify_dir = '/Volumes/Data2/RST/notebook/GTFS/TTC_2016-10-03/GIS Simplify/'
gtfs_dir = '/Volumes/Data2/RST/notebook/GTFS/TTC_2016-10-03/'

In [3]:
simplify_dir = 'D:/RST/RST/notebook/GTFS/TTC_2016-10-03/GIS Simplify/'

gtfs_dir = 'D:/RST/notebook/GTFS/TTC_2016-10-03/'

In [4]:
ward_int = pd.read_csv('D:/RST/notebook/GIS/int_wards.csv')
ward_int['INT_ID_STR'] = ward_int['INT_ID'].astype(str)

In [5]:
period = 'PM'
period_hr = 17

In [6]:
G = nx.read_gexf('D:/RST/notebook/networks/' + period +'-RM-16-singleDi.gexf')
graph_data = pd.read_csv('D:/RST/notebook/networks/' + period +'-RM-16-singleDi.csv')
stop_times_od = pd.read_csv(gtfs_dir + 'stop_times_full.csv')

In [7]:
route_dir = stop_times_od[stop_times_od['hr_o'] == period_hr]
route_dir = route_dir[['INT_ID_o', 'INT_ID_d', 'route_short_name']].drop_duplicates()
route_dir = route_dir.groupby(['INT_ID_o', 'INT_ID_d'])['route_short_name'].apply(list).reset_index()
route_dir = route_dir.merge(graph_data)

In [8]:
def func(row):

    single_cycle = row.copy()
    single_cycle.append(single_cycle[0])
    single_cycle.append(None)


    cycle_pairs = list(zip(single_cycle [::1], single_cycle [1::1]))[:-1]
    cycle_pairs = [list(map(int, sublist)) for sublist in cycle_pairs]
    cycle_df = route_dir.merge(pd.DataFrame.from_records(cycle_pairs, columns = ['INT_ID_o', 'INT_ID_d']))

    first_route = cycle_df.iloc[0,2]

    #check for cycles where it can be traversed by 1 route, ie eliminate out and back trips
    for route in first_route:
        if len(cycle_df[cycle_df['route_short_name'].apply(lambda x: route in x)]) == len(cycle_df):
            return ward,0,0,1
            break
        else:
            return ward, 1, cycle_df['scaled_freq'].min(), 0

In [9]:
result_list = []
ward_n = []

for ward in range(1,26,1):

    G_ward = G.copy()


    G_ward.remove_nodes_from(list(ward_int[ward_int['AREA_SHORT_CODE'] != ward]['INT_ID_STR']))

    N = G_ward.size()

    #find cycles
    cycle_list = list(nx.simple_cycles(G_ward))
    print('Ward ' + str(ward) + ', Size ' + str(N) + ' Found '+ str(len(cycle_list)) + ' cycles')

    temp_list = list(map(func, cycle_list))

    result_list.append(temp_list)
    ward_n.append([ward, N])


Ward 1, Size 383 Found 173253 cycles
Ward 2, Size 324 Found 263 cycles
Ward 3, Size 414 Found 925094 cycles
Ward 4, Size 216 Found 3166 cycles
Ward 5, Size 368 Found 341594 cycles
Ward 6, Size 304 Found 6534 cycles
Ward 7, Size 292 Found 10190 cycles
Ward 8, Size 275 Found 1422 cycles
Ward 9, Size 187 Found 351 cycles
Ward 10, Size 135 Found 398 cycles
Ward 11, Size 190 Found 4253 cycles
Ward 12, Size 158 Found 89 cycles
Ward 13, Size 148 Found 1261 cycles
Ward 14, Size 267 Found 10534 cycles
Ward 15, Size 251 Found 154 cycles
Ward 16, Size 197 Found 793 cycles
Ward 17, Size 207 Found 360 cycles
Ward 18, Size 209 Found 358 cycles
Ward 19, Size 239 Found 171 cycles
Ward 20, Size 321 Found 2267 cycles
Ward 21, Size 279 Found 5198 cycles
Ward 22, Size 228 Found 37399 cycles
Ward 23, Size 269 Found 67990 cycles
Ward 24, Size 146 Found 76 cycles
Ward 25, Size 265 Found 126 cycles


In [10]:
combined = list(itertools.chain.from_iterable(result_list))
result_df = pd.DataFrame.from_records(
    combined, columns = ['ward', 'cycles', 'scaled_cycles', 'removed_cycles']).groupby(
    'ward').sum().reset_index()
result_df = result_df.merge(pd.DataFrame.from_records(ward_n, columns = ['ward', 'N']))
result_df

,ward,cycles,scaled_cycles,removed_cycles,N
0,1,173079,4813.8375,174,383
1,2,108,5.8750,155,324
2,3,924910,23247.3375,184,414
3,4,3068,88.5875,98,216
4,5,341436,10556.8500,158,368
5,6,6406,208.2125,128,304
6,7,10081,264.6000,109,292
7,8,1288,34.4625,134,275
8,9,263,11.3500,88,187
9,10,345,13.5125,53,135


In [11]:
result_df['cycle_availability'] = result_df['scaled_cycles']/(2 * result_df['N'] + 5)

In [12]:
result_df = result_df.merge(ward_int[['AREA_NAME', 'AREA_SHORT_CODE']].drop_duplicates(), left_on = ['ward'], 
                 right_on = ['AREA_SHORT_CODE']).rename(columns = {'AREA_NAME':'ward_name'}).drop(
    columns = ['AREA_SHORT_CODE'])
result_df.sort_values(by = 'cycle_availability', ascending = False)

,ward,cycles,scaled_cycles,removed_cycles,N,cycle_availability,ward_name
2,3,924910,23247.3375,184,414,27.907968,Etobicoke-Lakeshore
4,5,341436,10556.8500,158,368,14.246761,York South-Weston
0,1,173079,4813.8375,174,383,6.243628,Etobicoke North
22,23,67875,1789.0750,115,269,3.294797,Scarborough North
21,22,37292,953.3875,107,228,2.068086,Scarborough-Agincourt
13,14,10409,436.7125,125,267,0.810227,Toronto-Danforth
6,7,10081,264.6000,109,292,0.449236,Humber River-Black Creek
20,21,5070,222.6750,128,279,0.395515,Scarborough Centre
10,11,4169,142.8250,84,190,0.370974,University-Rosedale
5,6,6406,208.2125,128,304,0.339662,York Centre


In [13]:
result_df.to_csv('cycles-' + period + '.csv', index = False)